<h1>Find focus plane and store a copy of your in-focus images</h1>

The following notebook scans your images folder, finds the z-stack with the most organoids in focus and stores a copy of said z-stack under <code>./output/in_focus_organoids</code>.

It does so by using the ObjectSegmenter and ObjectClassifier files you obtained during training in the [1_train_and_setup](./1_train_and_setup.ipynb) Jupyter notebook. If you have not trained your own segmenter and classifiers it will use the ones contained in <code>./pretrained_APOC</code> by default.

In [ ]:
import pandas as pd
import os 

from utils import read_images, find_focus, find_highest_infocus, store_imgs

# Initialize GPU-acceleration if available
import pyclesperanto_prototype as cle  # version 0.24.1
device = cle.select_device("TX")
print("Used GPU: ", device)

<h2>Where are your files stored?</h2>    

Below you should **edit the directory** containing your acquired images (z-stacks). Easiest thing will be to copy the folder containing your images inside the <code>./data</code> directory.

In [ ]:
# Define the directory containing your files
directory_path ="./data/test_data/"

# The following function will read all the images contained within the directory_path above
# and store them grouped by well_id.
images_per_well = read_images(directory_path)

In [ ]:
# This is just a check, it prints the grouped z-stacks
for well_id, files in images_per_well.items():
    print(f'Well {well_id}:')
    for file_path in files:
        print(file_path)

<h2>Analyze your images</h2>

The next cell will analyze your folder of images and compute the number of organoids in focus per image, it might take around 10 minutes if you do not have a GPU in your computer. Just be patient, there is a progress bar FYI.

In [ ]:
nr_infocus_organoids = find_focus(images_per_well)

Below we will generate a pandas Dataframe that will contain the % of organoids in focus in each of our images and store it as a .csv file so we can load it again in case needed.

In [ ]:
# Create a Pandas DataFrame
df = pd.DataFrame(nr_infocus_organoids)

# Specify the directory path
directory = "./output"

# Check if the output directory already exists and create it if it does not
try:
    os.mkdir(directory)
    print(f"Directory '{directory}' created successfully.")
except FileExistsError:
    print(f"Directory '{directory}' already exists.")

# Save the DataFrame as a .csv file
df.to_csv('./output/Percentage_in_focus_per_well.csv', index=False)

# Display the DataFrame
df

In [ ]:
# Load the .csv and transform it into a dictionary again

# Load the CSV file into a DataFrame
df = pd.read_csv('./output/Percentage_in_focus_per_well.csv')

# Convert the DataFrame to a dictionary
nr_infocus_organoids = df.to_dict(orient='list')

Now we will choose the image with the most organoids in focus and store a copy under <code>./output/in_focus_organoids</code>. To do so first we extract the position of the stack with the highest % of in-focus organoids and then we copy the files.

In [ ]:
# Finding the z-stack with the most organoids in-focus
max_index_dict = find_highest_infocus(nr_infocus_organoids)
print(max_index_dict)

# In case one of the wells has no detectable organoids in focus, this will substitute the focal plane
# with an average of all focal planes in the plate

# Calculate the average of all values in the dictionary
average_value = round(sum(max_index_dict.values()) / len(max_index_dict))

# Substitute the 0 values for the average focal plane
for well, in_focus_stack in max_index_dict.items():
    if in_focus_stack == 0:
        max_index_dict[well] = average_value

print(max_index_dict)

In [ ]:
# Storing a copy of each z-stack with the most organoids in focus
store_imgs(images_per_well, max_index_dict)